In [ ]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [ ]:
from pathlib import Path
import time

import pandas as pd
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models

import metrics
import model
import datatset
import utils
from eval import Eval

In [ ]:
DATA_DIR = Path("/home/shaun/personal/kaggle-data/nybg2020/")

In [ ]:
RUN = 

In [ ]:
checkpoints_dir = DATA_DIR/f'models-{RUN}/'

In [ ]:
# load and update labels
df = pd.read_csv(DATA_DIR/"train/validation.csv")
df = df[["file_name", "category_id"]]

In [ ]:
# Update class labels
classes_map = utils.load_json(DATA_DIR/"classes-map.json")
new_labels = [classes_map[str(i)] for i in df["category_id"]]
df["new_labels"] = new_labels
df["augment"] = np.zeros(len(df)) # no augmentation during eval

In [ ]:
ds = datatset.HerbDataSet(df, DATA_DIR/"train", 256, label="new_labels")
loader = DataLoader(ds, batch_size=16, shuffle=True)

In [ ]:
# Metrics
acc = metrics.Accuracy()
f1 = metrics.F1()
writer = SummaryWriter(DATA_DIR/f'logs/{RUN}/validation')
manager = metrics.MetricManager([acc, f1], writer)

# Loss
criterion = nn.CrossEntropyLoss()

In [ ]:
# Model
m = model.get_model(models.resnet50(pretrained=False), p1=0, p2=0)

In [ ]:
Eval(manager, criterion, loader, m, checkpoints_dir).run()